# Advantages of using a custon chat GPT

- Increased security but enabling authentication to access company chat
- Improved user experience by creating use cases specific to a company need
- Greater flexifiblity by adding custom fallbacks to user inputs

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False
)
while True:
    content = input("Please enter your prompt")
    if content in ["quit","exit", "bye"]:
        print("goodbye!")
        break
    response = chain.invoke({"content":content})
    print(response)
    

C:\Users\Vagner Machado\AppData\Local\Temp\ipykernel_28828\2034590030.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(


Please enter your prompt What makes the sky blue during the day and red around dusk and dawn?


{'content': 'What makes the sky blue during the day and red around dusk and dawn?', 'text': "During the day, the sky appears blue because of the way sunlight is scattered in the Earth's atmosphere. The gases and particles in the atmosphere scatter sunlight in all directions, but shorter (blue) wavelengths are scattered more than longer (red) wavelengths due to Rayleigh scattering. This causes the blue light to be scattered all around the sky, making it appear blue to our eyes.\n\nDuring sunrise and sunset, the sunlight has to pass through a larger portion of the Earth's atmosphere compared to when the sun is directly overhead. This means that more of the shorter blue and green wavelengths are scattered out, and the longer red and orange wavelengths are able to reach our eyes more directly. This is why the sky appears red or orange during these times, as the shorter wavelengths are scattered away and the longer wavelengths dominate the colors we see."}


Please enter your prompt So what can be done to change the colors?


{'content': 'So what can be done to change the colors?', 'text': 'To change the colors of what specifically? Can you provide more context or details so I can assist you better?'}


Please enter your prompt bye


goodbye!


## As you can see there is no memory from one question to another!

In [ ]:
# Before workin on that , lets test the System Message

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        # SystemMessage(content="You are a chatbot having a conversation with a human"),
        SystemMessage(content="You are a chatbot having a conversation with a human. Respond only in French."),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False
)
while True:
    content = input("Please enter your prompt")
    if content in ["quit","exit", "bye"]:
        print("Gboodbye!")
        break
    response = chain.invoke({"content":content})
    print(response)

Please enter your prompt What are the top 5 most delicious desserts in Spanish cousine?


{'content': 'What are the top 5 most delicious desserts in Spanish cousine?', 'text': "Les 5 desserts les plus délicieux de la cuisine espagnole sont la crème catalane, les churros, le flan, la tarta de Santiago et les pestiños. Avez-vous déjà goûté l'un de ces desserts espagnols?"}


Please enter your prompt quit


goodbye!


# Adding Chat Memory Using ConversationBufferMemory

In [12]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory # To enable buffer
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder # last param added

In [14]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        # SystemMessage(content="You are a chatbot having a conversation with a human"),
        SystemMessage(content="You are a chatbot having a conversation with a human. Respond only in Spanish."),
        MessagesPlaceholder(variable_name="chat_history"),   # Injectd this here
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)
while True:
    content = input("Please enter your prompt")
    if content in ["quit","exit", "bye"]:
        print("Goodbye!")
        break
    response = chain.invoke({"content":content})
    print(response)

# As seen below the chat has history and can use that as the context.

Please enter your prompt what is the most common gas in earth


{'content': 'what is the most common gas in earth', 'chat_history': [HumanMessage(content='what is the most common gas in earth', additional_kwargs={}, response_metadata={}), AIMessage(content='El gas más común en la Tierra es el nitrógeno, que constituye aproximadamente el 78% de la atmósfera terrestre. ¡Es fundamental para mantener el equilibrio de nuestro planeta!', additional_kwargs={}, response_metadata={})], 'text': 'El gas más común en la Tierra es el nitrógeno, que constituye aproximadamente el 78% de la atmósfera terrestre. ¡Es fundamental para mantener el equilibrio de nuestro planeta!'}


Please enter your prompt what is the diameter


{'content': 'what is the diameter', 'chat_history': [HumanMessage(content='what is the most common gas in earth', additional_kwargs={}, response_metadata={}), AIMessage(content='El gas más común en la Tierra es el nitrógeno, que constituye aproximadamente el 78% de la atmósfera terrestre. ¡Es fundamental para mantener el equilibrio de nuestro planeta!', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the diameter', additional_kwargs={}, response_metadata={}), AIMessage(content='El diámetro de la Tierra es de aproximadamente 12,742 kilómetros. ¡Nuestro planeta es realmente grande e impresionante!', additional_kwargs={}, response_metadata={})], 'text': 'El diámetro de la Tierra es de aproximadamente 12,742 kilómetros. ¡Nuestro planeta es realmente grande e impresionante!'}


Please enter your prompt quit


Goodbye!


# Saving Chat Sessions

To accomplish this feature, we will save our chat history to a file ( which can also be saved to a database) just before exiting the application. 
When the application is started again, those files (or database entries) are read and populated - giving the session history capability to the user. Lets keep on building the above app with memory, note new additions are commented

In [17]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory, FileChatMessageHistory # Added last parameter so to enable file as data sorce for sessions.
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

In [ ]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

history = FileChatMessageHistory('chat_history_session.json') # create the file 

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    chat_memory = history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        # SystemMessage(content="You are a chatbot having a conversation with a human"),
        SystemMessage(content="You are a chatbot having a conversation with a human. Respond only in Spanish."),
        MessagesPlaceholder(variable_name="chat_history"),   # Injectd this here
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)
while True:
    content = input("Please enter your prompt")
    if content in ["quit","exit", "bye"]:
        print("Goodbye!")
        break
    response = chain.invoke({"content":content})
    print(response)

# As seen below the chat has history and can use that as the context.